# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [6]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [7]:
### General parameters
max_steps = 4000000 # Set maximum number of steps to run environment.
run_path = "mousegame2" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "mousegame2" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 5000 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 512 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'learning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [8]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Academy' started successfully!


Unity Academy name: Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 4
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: , 


### Train the Agent(s)

In [9]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-2000001.cptk


INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-2000001.cptk


Step: 2010000. Mean Reward: 0.7590460878885231. Std of Reward: 2.330375644435469.
Step: 2020000. Mean Reward: 1.0625059382422677. Std of Reward: 2.835183055849889.
Step: 2030000. Mean Reward: 1.4984731774415225. Std of Reward: 3.5965888632815766.
Step: 2040000. Mean Reward: 1.031648998822131. Std of Reward: 2.7177402818477208.
Step: 2050000. Mean Reward: 1.3526832641770234. Std of Reward: 2.933627703024844.
Saved Model
Step: 2060000. Mean Reward: 1.439150417827281. Std of Reward: 3.2033557834596462.
Step: 2070000. Mean Reward: 1.715378787878768. Std of Reward: 3.6239082379560172.
Step: 2080000. Mean Reward: 1.8124336973478719. Std of Reward: 3.578892577353645.
Step: 2090000. Mean Reward: 1.420124137931018. Std of Reward: 3.1107537813266117.
Step: 2100000. Mean Reward: 1.3666897506925053. Std of Reward: 2.9093550009364075.
Saved Model
Step: 2110000. Mean Reward: 1.9821943048575976. Std of Reward: 3.6651635472706983.
Step: 2120000. Mean Reward: 1.9272861842105022. Std of Reward: 3.649595

Step: 2990000. Mean Reward: 4.631902017291004. Std of Reward: 5.284231162240113.
Step: 3000000. Mean Reward: 5.001509433962197. Std of Reward: 5.286189159954077.
Saved Model
Step: 3010000. Mean Reward: 6.595567375886442. Std of Reward: 6.158828397615511.
Step: 3020000. Mean Reward: 6.416561403508691. Std of Reward: 6.038114620976761.
Step: 3030000. Mean Reward: 5.428620689655102. Std of Reward: 5.811981892667911.
Step: 3040000. Mean Reward: 5.585324675324603. Std of Reward: 5.745057974324664.
Step: 3050000. Mean Reward: 5.238132530120415. Std of Reward: 5.967453656213968.
Saved Model
Step: 3060000. Mean Reward: 4.061653543307034. Std of Reward: 4.910829015833206.
Step: 3070000. Mean Reward: 3.183187066974554. Std of Reward: 4.3848442007412896.
Step: 3080000. Mean Reward: 3.791091811414344. Std of Reward: 5.101952460304075.
Step: 3090000. Mean Reward: 4.92497058823523. Std of Reward: 5.5195132190805385.
Step: 3100000. Mean Reward: 5.218079268292617. Std of Reward: 5.793106734737175.
Sav

Step: 3980000. Mean Reward: 7.867119999999903. Std of Reward: 6.421149842948619.
Step: 3990000. Mean Reward: 7.424671814671723. Std of Reward: 6.441655776226925.
Step: 4000000. Mean Reward: 7.8441935483870004. Std of Reward: 6.251420754229324.
Saved Model
Saved Model
INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-4000001.cptk


INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-4000001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [10]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-4000001.cptk


INFO:tensorflow:Restoring parameters from ./models/mousegame2\model-4000001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
